In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
from policy import ConvNet, ResNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import numpy as np
from env import gogame
import tensorflow as tf
from utils import concept_folder_setup_and_score

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 7
board_name = f'{board_size}x{board_size}'
agents_to_sample = [0, 10, 20, 60, 100, 500]
resnet = True
random_subpar_rollout = True

model_type = "resnet" if resnet else "convnet"

# Mcts simulations
simulations = 2000

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

# Search for the model with the most seen games and use it for mcts simulations
folders = os.listdir(full_model_path)

# Sort the folders by the number in the name
sorted_folders = sorted(folders, key=lambda x: int(x.split('_')[-1].strip('.keras')))

# Get the last folder
mcts_model_path = full_model_path + sorted_folders[-1]

init_state, concept_type_single, concept_name = DynamicConcepts.opening_play(board_size)

print("Initial state:\n", gogame.str(init_state))

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path, random_subpar_rollout, resnet)

Initial state:
 	0 1 2 3 4 5 6 
0	╔═╤═╤═╤═╤═╤═╗
1	╟─┼─┼─┼─┼─┼─╢
2	╟─┼─┼─┼─┼─┼─╢
3	╟─┼─┼─┼─┼─┼─╢
4	╟─┼─┼─┼─┼─┼─╢
5	╟─┼─┼─┼─┼─┼─╢
6	╚═╧═╧═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 0, White Area: 0

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp_ijqv1x2/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp_ijqv1x2/assets


In [68]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [69]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 7, 7)
Negative cases:  (17, 5, 7, 7)


In [236]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [36]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [62]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [70]:
# Probing
BINARY = True
epochs = 100

In [73]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs,
    dynamic=True
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('dynamic', model_type, board_name, session_name, concept_name, name, score)

Positions to consider:  28
Epoch 1/100


2024-02-19 16:58:37.738280: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 1s/step - loss: 0.9240
Epoch 2/100
1/1 [==============================] - 0s 143ms/step - loss: 0.9232
Epoch 3/100
1/1 [==============================] - 0s 69ms/step - loss: 0.9223
Epoch 4/100
1/1 [==============================] - 0s 151ms/step - loss: 0.9215
Epoch 5/100
1/1 [==============================] - 0s 90ms/step - loss: 0.9207
Epoch 6/100
1/1 [==============================] - 0s 65ms/step - loss: 0.9198
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9190
Epoch 8/100
1/1 [==============================] - 0s 24ms/step - loss: 0.9182
Epoch 9/100
1/1 [==============================] - 0s 170ms/step - loss: 0.9173
Epoch 10/100
1/1 [==============================] - 0s 74ms/step - loss: 0.9165
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9157
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 0.9149
Epoch 13/100
1/1 [==============================] - 0s 28ms/

In [74]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs,
            dynamic=True
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {concept_name} in resblock {i} is {score}")
    
    concept_presences[concept_name] = concept_presence_per_layer

    concept_folder_setup_and_score('dynamic', model_type, board_name, session_name, concept_name, epoch, concept_presence_per_layer)

Getting activation outputs: 100%|██████████| 1/1 [00:00<00:00,  9.66it/s]


Performing regression for layer 0
Epoch 1/100


2024-02-19 16:58:46.727196: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 2s 2s/step - loss: 1.2974
Epoch 2/100
1/1 [==============================] - 0s 226ms/step - loss: 1.2949
Epoch 3/100
1/1 [==============================] - 0s 224ms/step - loss: 1.2925
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 1.2900
Epoch 5/100
1/1 [==============================] - 0s 52ms/step - loss: 1.2876
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2852
Epoch 7/100
1/1 [==============================] - 0s 82ms/step - loss: 1.2827
Epoch 8/100
1/1 [==============================] - 0s 46ms/step - loss: 1.2803
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 1.2779
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 1.2755
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - loss: 1.2732
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 1.2708
Epoch 13/100
1/1 [==============================] - 0s 25ms/s

2024-02-19 16:58:51.695901: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

1/1 [==============================] - 1s 1s/step - loss: 2.4428
Epoch 2/100
1/1 [==============================] - 0s 206ms/step - loss: 2.3274
Epoch 3/100
1/1 [==============================] - 0s 100ms/step - loss: 2.2201
Epoch 4/100
1/1 [==============================] - 0s 187ms/step - loss: 2.1216
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 2.0328
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 1.9543
Epoch 7/100
1/1 [==============================] - 0s 123ms/step - loss: 1.8867
Epoch 8/100
1/1 [==============================] - 0s 23ms/step - loss: 1.8299
Epoch 9/100
1/1 [==============================] - 0s 51ms/step - loss: 1.7834
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.7458
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - loss: 1.7155
Epoch 12/100
1/1 [==============================] - 0s 53ms/step - loss: 1.6905
Epoch 13/100
1/1 [==============================] - 0s 32ms

2024-02-19 17:02:27.550991: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 991ms/step - loss: 1.2766
Epoch 2/100
1/1 [==============================] - 0s 241ms/step - loss: 1.2738
Epoch 3/100
1/1 [==============================] - 0s 58ms/step - loss: 1.2710
Epoch 4/100
1/1 [==============================] - 0s 96ms/step - loss: 1.2682
Epoch 5/100
1/1 [==============================] - 0s 182ms/step - loss: 1.2654
Epoch 6/100
1/1 [==============================] - 0s 187ms/step - loss: 1.2626
Epoch 7/100
1/1 [==============================] - 0s 51ms/step - loss: 1.2598
Epoch 8/100
1/1 [==============================] - 0s 70ms/step - loss: 1.2571
Epoch 9/100
1/1 [==============================] - 0s 56ms/step - loss: 1.2543
Epoch 10/100
1/1 [==============================] - 0s 171ms/step - loss: 1.2515
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2488
Epoch 12/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2461
Epoch 13/100
1/1 [==============================] - 0s 2

2024-02-19 17:02:32.147333: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 1s/step - loss: 0.8355
Epoch 2/100
1/1 [==============================] - 0s 159ms/step - loss: 0.8353
Epoch 3/100
1/1 [==============================] - 0s 190ms/step - loss: 0.8351
Epoch 4/100
1/1 [==============================] - 0s 117ms/step - loss: 0.8348
Epoch 5/100
1/1 [==============================] - 0s 195ms/step - loss: 0.8346
Epoch 6/100
1/1 [==============================] - 0s 53ms/step - loss: 0.8344
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 0.8342
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 0.8340
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 0.8338
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 0.8336
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - loss: 0.8334
Epoch 12/100
1/1 [==============================] - 0s 24ms/step - loss: 0.8332
Epoch 13/100
1/1 [==============================] - 0s 24ms